In [7]:
import MetaTrader5 as mt5 
import pandas as pd
from datetime import datetime
mt5.initialize()
# Replace following with your MT5 Account Login
account=51434456 # 
password="9UpBvVzc"
server = 'ICMarkets-Demo'

In [22]:
from_date=datetime(2023,10,19)
to_date=datetime.now()

In [30]:
history = mt5.history_deals_get(from_date, to_date)

In [31]:
positions = []

In [35]:
for pos in history:
    if 'Regress' or 'MC' in pos.comment:
        if pos.volume < 2.5:
            positions.append(pos)

In [36]:
history = []

In [37]:
positions

[TradeDeal(ticket=365981534, order=503625247, time=1697677425, time_msc=1697677425314, type=1, entry=0, magic=234000, position_id=503625247, reason=3, volume=1.0, price=1.21426, commission=-3.5, swap=0.0, profit=0.0, fee=0.0, symbol='GBPUSD.a', comment='S_Regress', external_id=''),
 TradeDeal(ticket=365981535, order=503625248, time=1697677425, time_msc=1697677425791, type=1, entry=0, magic=234000, position_id=503625248, reason=3, volume=1.0, price=2.07369, commission=-3.5, swap=0.0, profit=0.0, fee=0.0, symbol='GBPNZD.a', comment='S_Regress', external_id=''),
 TradeDeal(ticket=365981537, order=503625250, time=1697677426, time_msc=1697677426261, type=0, entry=0, magic=234000, position_id=503625250, reason=3, volume=1.15, price=1.05377, commission=-4.03, swap=0.0, profit=0.0, fee=0.0, symbol='EURUSD.a', comment='B_Regress', external_id=''),
 TradeDeal(ticket=365981538, order=503625256, time=1697677426, time_msc=1697677426731, type=0, entry=0, magic=234000, position_id=503625256, reason=3

In [28]:
for pos in positions:
    history.append(pos.symbol)
    history.append(pos.comment)
    history.append(pos.profit)

AttributeError: 'TradeOrder' object has no attribute 'profit'